<a href="https://colab.research.google.com/github/MAHMUD722025/KNn-Book-recomandation/blob/main/KNN_Book_Recomandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ================================
# Book Recommendation using KNN
# ================================
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# 1. Load Dataset
books = pd.read_csv("/content/drive/MyDrive/BX-Books.csv", sep=';', on_bad_lines='skip', encoding='latin-1')
ratings = pd.read_csv("/content/drive/MyDrive/BX-Book-Ratings.csv", sep=';', on_bad_lines='skip', encoding='latin-1')
users = pd.read_csv("/content/drive/MyDrive/BX-Users.csv", sep=';', on_bad_lines='skip', encoding='latin-1')

# 2. Keep useful columns only
books = books[['ISBN', 'Book-Title', 'Book-Author']]
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]

# 3. Filter out inactive users & unpopular books
# (Users with few ratings and books with very few readers বাদ দিব)
ratings_count = ratings['User-ID'].value_counts()
ratings = ratings[ratings['User-ID'].isin(ratings_count[ratings_count >= 200].index)]

book_count = ratings['ISBN'].value_counts()
ratings = ratings[ratings['ISBN'].isin(book_count[book_count >= 100].index)]

# 4. Merge Ratings + Books
df = ratings.merge(books, on='ISBN')

# 5. Create Pivot Table (User vs Book Rating Matrix)
pivot_table = df.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

# 6. Convert to Sparse Matrix (for memory efficiency)
matrix = csr_matrix(pivot_table.values)

# 7. KNN Model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matrix)

# 8. Function to Recommend Books
def recommend_book(book_name, n_recommendations=5):
    if book_name not in pivot_table.index:
        print("❌ Book not found in dataset")
        return

    book_index = pivot_table.index.get_loc(book_name)
    distances, indices = model.kneighbors([pivot_table.iloc[book_index]], n_neighbors=n_recommendations+1)

    print(f"📚 Because you liked '{book_name}', you may also like:")
    for i in indices[0][1:]:  # skip the first (itself)
        print(f"➡ {pivot_table.index[i]}")

# 9. Example Usage
recommend_book("1st to Die: A Novel")

/tmp/ipython-input-4006472551.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/content/drive/MyDrive/BX-Books.csv", sep=';', on_bad_lines='skip', encoding='latin-1')


📚 Because you liked '1st to Die: A Novel', you may also like:
➡ Along Came a Spider (Alex Cross Novels)
➡ Two for the Dough
➡ One for the Money (Stephanie Plum Novels (Paperback))
➡ Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)
➡ Kiss the Girls


In [ ]:
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Map of the World,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Painted House,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Prayer for Owen Meany,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Where the Heart Is (Oprah's Book Club (Paperback)),0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
While I Was Gone,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
White Oleander : A Novel (Oprah's Book Club),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(pivot_table.index[:10])

Index(['1st to Die: A Novel',
       'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
       'A Map of the World', 'A Painted House', 'A Prayer for Owen Meany',
       'A Time to Kill', 'A Walk to Remember', 'All I Really Need to Know',
       'Along Came a Spider (Alex Cross Novels)', 'Angels &amp; Demons'],
      dtype='object', name='Book-Title')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
